In [1]:
from __future__ import print_function

import argparse
import os
import shutil
import time
import random
import math
import numpy
from tqdm import tqdm
from torchvision import models, datasets, transforms
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
# import models.cifar as models
import dataset_utils

In [2]:
 dataset = datasets.CIFAR10
num_classes = 10
input_size = 32
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])
trainset = dataset(root='./data', train=True, download=True, transform=transform_train)
testset = dataset(root='./data', train=False, download=True, transform=transform_test)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
 dataset = datasets.SVHN
num_classes = 10
input_size = 32
transform_train = transforms.Compose([
        transforms.RandomCrop(32,padding=4),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
train_dataset = dataset(root='./data', split='train', transform=transform_train,
                                 target_transform=None, download=True)
test_dataset = dataset(root='./data', split='test', transform=transform_test,
                               target_transform=None, download=True)


Using downloaded and verified file: ./data\train_32x32.mat
Using downloaded and verified file: ./data\test_32x32.mat


In [ ]:
num_classes = 2
input_size = 64
transform_train = transforms.Compose([
    transforms.RandomCrop(64, padding=6),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
# use the "ImageFolder" datasets
assert os.path.exists("./data/cats_dogs/train") and os.path.exists("./data/cats_dogs/test"), "Please download and put the 'cats vs dogs' dataset to paths 'data/cats_dogs/train' and 'cats_dogs/test'"
trainset =  datasets.ImageFolder('./data/cats_dogs/train')
testset =  datasets.ImageFolder('./data/cats_dogs/test')
# resizing the images to 64 and center crop them, so that they become 64x64 squares
trainset = dataset_utils.resized_dataset(trainset, transform_train, resize=64)
testset = dataset_utils.resized_dataset(testset, transform_test, resize=64)

In [81]:
train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=4)

NameError: name 'trainset' is not defined

In [ ]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()

#         self.resnet = models.resnet18(pretrained=False, num_classes=10)

#         self.resnet.conv1 = torch.nn.Conv2d(
#             3, 64, kernel_size=3, stride=1, padding=1, bias=False
#         )
#         self.resnet.maxpool = torch.nn.Identity()
        
        self.resnet = models.vgg16(pretrained=False, num_classes=10+1)
    def forward(self, x):
        x = self.resnet(x)
#         x = F.log_softmax(x, dim=1)

        return x

In [ ]:
use_cuda=True
# Model
print("==> creating model '{}'".format('vgg16'))
# model = models.__dict__['vgg16'](num_classes=num_classes+1, input_size = input_size)
model=Model()
# 'alexnet': <function models.cifar.alexnet.alexnet(**kwargs)>,
#  'vgg': <module 'models.cifar.vgg' from 'C:\\Users\\Student\\models\\cifar\\vgg.py'>,
#  'VGG': models.cifar.vgg.VGG,
#  'vgg16': <function models.cifar.vgg.vgg16(**kwargs)>,
#  'vgg16_bn': <function models.cifar.vgg.vgg16_bn(**kwargs)>,
#  'resnet': <function models.cifar.resnet.resnet(**kwargs)>,
#  'resnext': <function models.cifar.resnext.resnext(**kwargs)>,
#  'wrn': <function models.cifar.wrn.wrn(**kwargs)>,
#  'preresnet': <function models.cifar.preresnet.preresnet(**kwargs)>,
#  'densenet': <function models.cifar.densenet.densenet(**kwargs)>}
if use_cuda: model = torch.nn.DataParallel(model.cuda())
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))
pretrain=10

if pretrain: criterion = nn.CrossEntropyLoss() 
# the conventional loss is replaced by the gambler's loss in train() and test() explicitly except for pretraining
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)


In [ ]:
gamma=0.5
schedule=[20,40,75]
state={}
state['lr']=0.001
def adjust_learning_rate(optimizer, epoch):
    global state
    if epoch in schedule:
        state['lr'] *= gamma
        for param_group in optimizer.param_groups:
            param_group['lr'] = state['lr']

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value
       Imported from https://github.com/pytorch/examples/blob/master/imagenet/main.py#L247-L262
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [ ]:
reward=2.2

In [ ]:
def train(trainloader, model, criterion, optimizer, epoch, use_cuda):
    # switch to train mode
    model.train()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    end = time.time()

    
    for batch_idx,  (inputs, targets) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
#         inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        if epoch >= pretrain:
            outputs = F.softmax(outputs, dim=1)
            outputs, reservation = outputs[:,:-1], outputs[:,-1]
            gain = torch.gather(outputs, dim=1, index=targets.unsqueeze(1)).squeeze()
            doubling_rate = (gain.add(reservation.div(reward))).log()

            loss = -doubling_rate.mean()
        else:
            loss = criterion(outputs[:,:-1], targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.item(), inputs.size(0))
        top1.update(prec1.item(), inputs.size(0))
        top5.update(prec5.item(), inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        progress  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Loss: {loss:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=batch_idx + 1,
                    size=len(trainloader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    )
    print(progress)
    return (losses.avg, top1.avg)

In [ ]:
def test(testloader, model, criterion, epoch, use_cuda, evaluation = False):
    global best_acc
    expected_coverage=[100.,99.,98.,97.,95.,]
    # whether to evaluate uncertainty, or confidence
    if evaluation:
        evaluate(testloader, model, use_cuda)
        return

    # switch to test mode
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    model.eval()

    end = time.time()
    reservation=[]
    abstention_results = []
    for batch_idx, (inputs, targets) in enumerate(testloader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        with torch.no_grad():
            outputs = model(inputs)
            if epoch >= pretrain:
                outputs = F.softmax(outputs, dim=1)
                outputs, reservation = outputs[:,:-1], outputs[:,-1]
                predictions = outputs.data.max(1)[1]
                # analyze the accuracy at different abstention level
                abstention_results.extend(zip(list( reservation.cpu() ),list( predictions.eq(targets.data).cpu() )))
                
                # calculate loss
                gain = torch.gather(outputs, dim=1, index=targets.unsqueeze(1)).squeeze()
                doubling_rate = (gain.add(reservation.div(reward))).log()
                loss = -doubling_rate.mean()
            else:
                loss = criterion(outputs[:,:-1], targets)
#             return outputs, reservation, abstention_results
            # measure accuracy and record loss
            prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
            losses.update(loss.item(), inputs.size(0))
            top1.update(prec1.item(), inputs.size(0))
            top5.update(prec5.item(), inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        res=abstention_results
        
        # plot progress
        suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | | Loss: {loss:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
                    batch=batch_idx + 1,
                    size=len(testloader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    loss=losses.avg,
                    top1=top1.avg,
                    top5=top5.avg,
                    )
    print(suffix)
    if epoch >= pretrain:
        # sort the abstention results according to their reservations, from high to low
        abstention_results.sort(key = lambda x: x[0])
        # get the "correct or not" list for the sorted results
        sorted_correct = list(map(lambda x: int(x[1]), abstention_results))
        size = len(sorted_correct)
        print('accracy of coverage ',end='')
        for coverage in expected_coverage:
            covered_correct = sorted_correct[:round(size/100*coverage)]
            print('{:.0f}: {:.3f}, '.format(coverage, sum(covered_correct)/len(covered_correct)*100.), end='')
        print('')
    return (losses.avg, top1.avg,res)



In [ ]:
outputs, reservation, abstention_resultsreservation=test_loss, test_acc, res

In [113]:
10%6

4

In [ ]:
evaluate=True
if evaluate:
    print('\nEvaluation only')
    test(testloader, model, criterion, 0, use_cuda, evaluation = True)
    

In [ ]:
epochs=10
for epoch in range(0, epochs):
        adjust_learning_rate(optimizer, epoch)
#         print('\n'+save_path)
#         print('Epoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))

        train_loss, train_acc = train(trainloader, model, criterion, optimizer, epoch, use_cuda)
        test_loss, test_acc, res = test(testloader, model, criterion, epoch, use_cuda)
        # save the model
#         filepath = os.path.join(save_path, "{:d}".format(epoch+1) + ".pth")
#         torch.save(model, filepath)
#         # delete the last saved model if exist
#         last_path = os.path.join(save_path, "{:d}".format(epoch) + ".pth")
#         if os.path.isfile(last_path): os.remove(last_path)
#         # append logger file
#         logger.append([epoch+1, state['lr'], train_loss, test_loss, 100-train_acc, 100-test_acc])

# filepath = os.path.join(save_path, "{:d}".format(args.epochs) + ".pth")
# torch.save(model, filepath)
# last_path = os.path.join(save_path, "{:d}".format(args.epochs-1) + ".pth")


In [3]:

def get_CIFAR10(root="./"):
    input_size = 32
    num_classes = 10
    normalize = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    
    train_transform = transforms.Compose(
        [
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]
    )
    train_dataset = datasets.CIFAR10(
        root + "data/CIFAR10", train=True, transform=train_transform, download=True
    )

    test_transform = transforms.Compose(
        [
            transforms.ToTensor(),
            normalize,
        ]
    )
    test_dataset = datasets.CIFAR10(
        root + "data/CIFAR10", train=False, transform=test_transform, download=True
    )

    return input_size, num_classes, train_dataset, test_dataset

def train(model, train_loader, optimizer, epoch,alpha=1.0):
    model.train()
    
    
#     const=1-epoch/50
    total_loss = []
    
    for data, target in train_loader:  #tqdm()
        data = data.cuda()
        target = target.cuda()

        optimizer.zero_grad()

        prediction = model(data)
        if epoch >= pretrain:
            outputs = F.softmax(prediction, dim=1)
            outputs, reservation = outputs[:,:-1], outputs[:,-1]
            gain = torch.gather(outputs, dim=1, index=target.unsqueeze(1)).squeeze()
            doubling_rate = (gain.add(reservation.div(reward))).log()

            loss = -doubling_rate.mean()
        else:
            
#             loss = F.cross_entropy(prediction[:,:-1], target) #cross_entropy  nll_loss
            
            loss = NotWrongLoss(prediction,target,alpha=alpha)
#             print(loss)
        loss.backward()
        optimizer.step()

        total_loss.append(loss.item())

    avg_loss = sum(total_loss) / len(total_loss)
    print(f"Epoch: {epoch}:")
    print(f"Train Set: Average Loss: {avg_loss:.2f}")
def test(model, test_loader):
    model.eval()

    loss = 0
    correct = 0
    abstention_results=[]
    for data, target in test_loader:
        with torch.no_grad():
            data = data.cuda()
            target = target.cuda()

            prediction = model(data)
            if epoch >= pretrain:
                outputs = F.softmax(prediction, dim=1)
                outputs, reservation = outputs[:,:-1], outputs[:,-1]
                predictions = outputs.data.max(1)[1]
                # analyze the accuracy at different abstention level
                abstention_results.extend(zip(list( reservation.cpu() ),list( predictions.eq(target.data).cpu() )))
                
                # calculate loss
                gain = torch.gather(outputs, dim=1, index=target.unsqueeze(1)).squeeze()
                doubling_rate = (gain.add(reservation.div(reward))).log()
                loss = -doubling_rate.mean()
            else:
                
                
                loss += F.nll_loss(prediction[:,:-1], target, reduction="sum")

            prediction = prediction.max(1)[1]
            correct += prediction.eq(target.view_as(prediction)).sum().item()

    loss /= len(test_loader.dataset)

    percentage_correct = 100.0 * correct / len(test_loader.dataset)

    print(
        "Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)".format(
            loss, correct, len(test_loader.dataset), percentage_correct
        )
    )
    if epoch >= pretrain:
        # sort the abstention results according to their reservations, from high to low
        abstention_results.sort(key = lambda x: x[0])
        # get the "correct or not" list for the sorted results
        sorted_correct = list(map(lambda x: int(x[1]), abstention_results))
        size = len(sorted_correct)
        print('accracy of coverage ',end='')
        expected_coverage=[100.,98.,95.]
        for coverage in expected_coverage:
            covered_correct = sorted_correct[:round(size/100*coverage)]
            print(covered_correct)
            print('{:.0f}: {:.3f}, '.format(coverage, sum(covered_correct)/len(covered_correct)*100.), end='')
        print('')
    
    return loss, percentage_correct
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()

#         self.resnet = models.resnet18(pretrained=False, num_classes=10)

#         self.resnet.conv1 = torch.nn.Conv2d(
#             3, 64, kernel_size=3, stride=1, padding=1, bias=False
#         )
#         self.resnet.maxpool = torch.nn.Identity()
        
        self.resnet = models.vgg16(pretrained=False, num_classes=10+1)
    def forward(self, x):
        x = self.resnet(x)
        x = F.log_softmax(x, dim=1)

        return x

In [4]:
###Not wrong loss
def NotWrongLoss(outputs,label,alpha=1.5):

    outputs=F.softmax(outputs, dim=1)
    # calculate loss
    gain = torch.gather(outputs, dim=1, index=label.unsqueeze(1)).squeeze()+outputs[:,-1]+1e-8
    NW_Loss = -gain.log()-alpha*torch.log(outputs[:,:-1].sum(1)+1e-8)
    
    return NW_Loss.mean()

In [8]:
# torch.manual_seed(1)#(args.seed)

# input_size, num_classes, train_dataset, test_dataset = get_CIFAR10()

kwargs = {"num_workers": 2, "pin_memory": True}

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=128, shuffle=True, **kwargs
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=128, shuffle=False, **kwargs
)


In [9]:
pretrain=100
reward=2
alpha=[0.5,1.0,1.5,2,4]
# model = models.__dict__['vgg16'](num_classes=num_classes, input_size = input_size)
# model=Model()
for a in alpha:
    for i in range(10):
        model = VGG(16, 11)#ResNet(18, 11)  #
        model.load_state_dict(torch.load(f"vgg16_svhn_model_{i}.pt"))
        model = model.cuda()

        milestones = [10,20,25,30,35, 40,45, 50, 55, 60,  70,  80,90]

        optimizer = torch.optim.SGD(
            model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4
        )
        scheduler = torch.optim.lr_scheduler.MultiStepLR(
            optimizer, milestones=milestones, gamma=0.5
        )

        for epoch in range(1, 10 + 1):
            torch.cuda.empty_cache()
            train(model, train_loader, optimizer, epoch,alpha=a)
            torch.cuda.empty_cache()
            test(model, test_loader)

            scheduler.step()

        torch.save(model, f"D:/OneDrive - Deakin University/review paper/data for paper/NWLoss/alpha {a}/NWLoss_vgg16_svhn_model_{i}.pt")

Epoch: 1:
Train Set: Average Loss: 0.01
Test set: Average loss: -7.9929, Accuracy: 25021/26032 (96.12%)
Epoch: 2:
Train Set: Average Loss: 0.01
Test set: Average loss: -7.9701, Accuracy: 25020/26032 (96.11%)
Epoch: 3:
Train Set: Average Loss: 0.01
Test set: Average loss: -8.1242, Accuracy: 25030/26032 (96.15%)
Epoch: 4:
Train Set: Average Loss: 0.01
Test set: Average loss: -8.0836, Accuracy: 25040/26032 (96.19%)
Epoch: 5:
Train Set: Average Loss: 0.01
Test set: Average loss: -8.1436, Accuracy: 25038/26032 (96.18%)
Epoch: 6:
Train Set: Average Loss: 0.01
Test set: Average loss: -8.1308, Accuracy: 25048/26032 (96.22%)
Epoch: 7:
Train Set: Average Loss: 0.01
Test set: Average loss: -8.1218, Accuracy: 25011/26032 (96.08%)
Epoch: 8:
Train Set: Average Loss: 0.01
Test set: Average loss: -8.1999, Accuracy: 25049/26032 (96.22%)
Epoch: 9:
Train Set: Average Loss: 0.01
Test set: Average loss: -8.1487, Accuracy: 25050/26032 (96.23%)
Epoch: 10:
Train Set: Average Loss: 0.01
Test set: Average loss:

In [29]:
covered_correct

NameError: name 'covered_correct' is not defined

In [8]:
model = VGG(16, 11)
model.load_state_dict(torch.load(f"vgg16_cifar_model_0.pt"))
model=model.cuda()

In [9]:
zipped=[]
correct=0
for data,label in test_loader:
    torch.cuda.empty_cache()
    with torch.no_grad():
        pre_softmax=model(data.cuda())  #
        softmax = F.softmax(pre_softmax, dim=1)
    softmax=softmax.cpu()            #[:,:-1]
    entropy=(softmax*torch.log(softmax)).sum(1)
    GI=1-torch.pow(softmax,2).sum(1)

    top=(softmax-softmax.max(1)[0].unsqueeze(1)).sum(1).pow(2)
    mu_2=(1-softmax.max(1)[0])/(11-1)
    down=torch.pow(torch.sort(softmax)[0][:,:-1]-mu_2.unsqueeze(1),2).sum(1)
    LDAM=2*top/((11-1)*down)

    sorted_softmax=torch.sort(softmax)[0]-softmax.max(1)[0].unsqueeze(1)
    J_P=softmax.max(1)[0]-sorted_softmax.var(1)-sorted_softmax.mean(1)
    
    confidence=1-torch.sort(softmax)[0][:,-2]


    sorted_softmax=torch.sort(softmax)[0][:,:]
    Relative_diff=1-sorted_softmax[:,-2]/sorted_softmax[:,-1]
    
    zipped.extend(list(zip(softmax[:,-1],GI,entropy,LDAM,J_P,confidence,Relative_diff,softmax.max(1)[1].eq(label))))
    



In [62]:
correct

0

In [63]:
name=['extra class ','GI\t\t','entropy\t','LDAM\t','J_P\t\t','confidence\t','Relativ_diff']

In [64]:
#which metric is needed higher value, that should be saved in descending order
print('\t\t\t\t\tcoverage\n\t\t\t     98\t\t  95\t       90')
for k in range(7):
    descending=True
    if k<2:
        descending=False#True#
    reverse=False
    if descending:
        reverse=True
    zipped.sort(key = lambda x: x[k],reverse=reverse)
    sorted_correct = list(map(lambda x: int(x[7]), zipped))
    size = len(sorted_correct)
    print(f'accracy for {name[k]} ',end='')
    expected_coverage=[98.,95.,90.]
    for coverage in expected_coverage:
        covered_correct = sorted_correct[:round(size/100*coverage)]
        print('{} {:.3f} |'.format(sum(covered_correct), sum(covered_correct)/len(covered_correct)*100.), end='')
    print('\n')

					coverage
			     98		  95	       90
accracy for extra class  24745 96.997 |24220 97.938 |23074 98.485 |

accracy for GI		 24758 97.048 |24281 98.184 |23140 98.766 |

accracy for entropy	 24762 97.064 |24278 98.172 |23136 98.749 |

accracy for LDAM	 24751 97.021 |24283 98.192 |23161 98.856 |

accracy for J_P		 24758 97.048 |24281 98.184 |23140 98.766 |

accracy for confidence	 24743 96.990 |24283 98.192 |23158 98.843 |

accracy for Relativ_diff 24752 97.025 |24282 98.188 |23158 98.843 |



In [46]:
print('\tcoverage\tcoverage\tcoverage\tcoverage')
for i in range(5):
    print('  {:.0f}:{} {:.3f}, '.format(coverage, sum(covered_correct), sum(covered_correct)/len(covered_correct)*100.), end='')


	coverage	coverage	coverage	coverage
  90:8428 93.644,   90:8428 93.644,   90:8428 93.644,   90:8428 93.644,   90:8428 93.644, 

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.init as init
import numpy as np
import pdb

def conv_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        init.xavier_uniform_(m.weight, gain=np.sqrt(2))
        init.constant_(m.bias, 0)

def cfg(depth):
    depth_lst = [11, 13, 16, 19]
    assert (depth in depth_lst), "Error : VGGnet depth should be either 11, 13, 16, 19"
    cf_dict = {
        '11': [
            64, 'mp',
            128, 'mp',
            256, 256, 'mp',
            512, 512, 'mp',
            512, 512, 'mp'],
        '13': [
            64, 64, 'mp',
            128, 128, 'mp',
            256, 256, 'mp',
            512, 512, 'mp',
            512, 512, 'mp'
            ],
        '16': [
            64, 64, 'mp',
            128, 128, 'mp',
            256, 256, 256, 'mp',
            512, 512, 512, 'mp',
            512, 512, 512, 'mp'
            ],
        '19': [
            64, 64, 'mp',
            128, 128, 'mp',
            256, 256, 256, 256, 'mp',
            512, 512, 512, 512, 'mp',
            512, 512, 512, 512, 'mp'
            ],
    }

    return cf_dict[str(depth)]

def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=True)

class VGG(nn.Module):
    def __init__(self, depth, num_classes, dropout_rate = 0.):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg(depth), dropout_rate)
        self.linear = nn.Linear(512, num_classes)

    def forward(self, x):
        out = self.features(x)

        #below to work with datasets that have resolution other than 32x32
        m = torch.nn.AdaptiveAvgPool2d((1))
        out = m(out)
        #  --end 

        #pdb.set_trace()
        out = out.view(out.size(0), -1)
        out = self.linear(out)

        return out

    def _make_layers(self, cfg, dropout_rate):
        layers = []
        in_planes = 3

        for x in cfg:
            if x == 'mp':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                if dropout_rate > 0.:
                    layers += [conv3x3(in_planes, x), nn.BatchNorm2d(x), 
                        nn.ReLU(inplace=True), nn.Dropout(p=dropout_rate)]
                else:
                    layers += [conv3x3(in_planes, x), nn.BatchNorm2d(x), nn.ReLU(inplace=True)]
                in_planes = x

        # After cfg convolution
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)



In [ ]:
model = VGG(16, 11)
y = net(Variable(torch.randn(1,3,32,32)))
print(y.size())

In [ ]:
print('    Total params: %.2fM' % (sum(p.numel() for p in net.parameters())/1000000.0))

In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.nn.init as init

from torch.autograd import Variable
import sys

def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=True)

def conv_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        init.xavier_uniform_(m.weight, gain=np.sqrt(2))
        init.constant_(m.bias, 0)

def cfg(depth):
    depth_lst = [18, 34, 50, 101, 152]
    assert (depth in depth_lst), "Error : Resnet depth should be either 18, 34, 50, 101, 152"
    cf_dict = {
        '18': (BasicBlock, [2,2,2,2]),
        '34': (BasicBlock, [3,4,6,3]),
        '50': (Bottleneck, [3,4,6,3]),
        '101':(Bottleneck, [3,4,23,3]),
        '152':(Bottleneck, [3,8,36,3]),
    }

    return cf_dict[str(depth)]

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(in_planes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=True),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)

        return out

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=True)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=True)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=True)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=True),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)

        return out

class ResNet(nn.Module):
    def __init__(self, depth, num_classes):
        super(ResNet, self).__init__()
        self.in_planes = 16

        block, num_blocks = cfg(depth)

        self.conv1 = conv3x3(3,16)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(64*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []

        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)

        return out


In [ ]:
net=ResNet(18, 10)
y = net(Variable(torch.randn(1,3,32,32)))
print(y.size())

In [ ]:
print('Total params: %.2fM' % (sum(p.numel() for p in net.parameters())/1000000.0))